In [1]:
import sys
sys.path.append('../')
from poker_env_multi import PokerEnvMulti
from agents.random_policy import RandomActions
from agents.heuristic_policy import HeuristicPolicy
from ray.rllib.algorithms.ppo import PPOConfig
from gym import spaces

import ray

from ray.rllib.policy.policy import PolicySpec
from ray.tune.registry import register_env

/Users/adamprice/Applications/anaconda3/envs/stocktake/lib/python3.8/site-packages/keras_preprocessing/image/utils.py:23: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
/Users/adamprice/Applications/anaconda3/envs/stocktake/lib/python3.8/site-packages/keras_preprocessing/image/utils.py:24: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': pil_image.BILINEAR,
/Users/adamprice/Applications/anaconda3/envs/stocktake/lib/python3.8/site-packages/keras_preprocessing/image/utils.py:25: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  'bicubic': pil_image.BICUBIC,
/Users/adamprice/Applications/anaconda3/envs/stocktake/lib/python3.8/site-packages/keras_preprocessing/image/utils.py:28: DeprecationWarning: HAMMING is dep

In [2]:
def env_creator(config):
    env = PokerEnvMulti(config)
    return env

register_env("poker", lambda config: env_creator(config))

/Users/adamprice/Applications/anaconda3/envs/stocktake/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
def select_policy(agent_id, episode, **kwargs):
    if agent_id == "player_0":
        return "random"
    elif agent_id == "player_1":
        return "Heuristic_10"
    elif agent_id == "player_2":
        return "Heuristic_100"
    elif agent_id == "player_3":
        return "Heuristic_1000"
    return "Heuristic_1000"

kwargs = {'test': 't'}
observation_space = spaces.Dict({
            "obs": spaces.Box(0, 500, shape=(29, )),
            "state": spaces.Box(0, 1, shape=(28, ))
        })
action_space = spaces.Discrete(3)

config = (
    PPOConfig()
    .rollouts(
        num_rollout_workers=0,
        num_envs_per_worker=1,
    )
    .training(
        train_batch_size=128,
        gamma=0.99,
    )
    .multi_agent(
        policies={
           "random": PolicySpec(policy_class=RandomActions),
           "Heuristic_10": (HeuristicPolicy, observation_space, action_space, {'difficulty': 0}),
           "Heuristic_100": (HeuristicPolicy, observation_space, action_space, {'difficulty': 1}),
           "Heuristic_1000": (HeuristicPolicy, observation_space, action_space, {'difficulty': 2}),
            #"learned": PolicySpec(
            #    config={
            #        "model": {"use_lstm": use_lstm},
            #        "framework": args.framework,
            #    }
            #),
        },
        policy_mapping_fn=select_policy,
        policies_to_train=[],
    )
    .resources(num_gpus=0)\
)
trainer = config.build(env="poker")


2022-11-05 12:23:01,108	WARNING multi_agent.py:121 -- `config.multiagent.policies_to_train` is empty! Make sure - if you would like to learn at least one policy - to add its ID to that list.
2022-11-05 12:23:01,111	WARNING ppo.py:350 -- `train_batch_size` (128) cannot be achieved with your other settings (num_workers=0 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 128.
/Users/adamprice/Applications/anaconda3/envs/stocktake/lib/python3.8/site-packages/ray/_private/ray_option_utils.py:266: DeprecationWarning: Setting 'object_store_memory' for actors is deprecated since it doesn't actually reserve the required object store memory. Use object spilling that's enabled by default (https://docs.ray.io/en/releases-2.0.1/ray-core/objects/object-spilling.html) instead to bypass the object store memory size limitation.
  warnings.warn(
2022-11-05 12:23:03,541	INFO worker.py:1518 -- Started a local Ray instance.
2022-11-05 12:23:05,445	WARNING env.p

In [7]:
import numpy as np
np.array([12, 18], dtype=np.int8).tobytes()

/Users/adamprice/Applications/anaconda3/envs/stocktake/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


b'\x0c\x12'

In [4]:
trainer.train()

[12 18]


KeyError: b'\x0c\x12'

In [ ]:
import numpy as np
winners = np.array([0,2])
losers = np.delete(np.arange(6), winners)
losers

In [ ]:
matrix = np.load('5_card_values_10.npy',allow_pickle=True)
hand = np.array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0])
matrix